In [5]:
import numpy as np
import pandas as pd
import time
# from tqdm import tqdm
import myslack_incomming
import scipy
import pickle
from sklearn.ensemble import GradientBoostingRegressor 

In [5]:
train = pickle.load(open("./data/imputation_train.pkl", "rb"))
test = pickle.load(open("./data/imputation_test.pkl", "rb"))

## Preprocessing Step2 -  Encoding

In [3]:
myslack_incomming.send_slack('Encoding started.')

<Response [200]>


In [4]:
train.tail()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
647049,39,191346,Sunday,3.239000e+10,1,PHARMACY OTC,1118.0
647050,39,191346,Sunday,7.874205e+09,1,FROZEN FOODS,1752.0
647051,39,191346,Sunday,4.072000e+03,1,PRODUCE,4170.0
647052,8,191347,Sunday,4.190008e+09,1,DAIRY,1512.0
647053,8,191347,Sunday,3.800060e+09,1,GROCERY DRY GOODS,3600.0


---

### 2-0-1. Train TripType

In [5]:
samsub = pd.read_csv('./data/sample_submission.csv')

In [6]:
samsub.tail()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
95669,191338,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95670,191339,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95671,191340,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95672,191341,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95673,191348,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train_TT_dummy_df = pd.get_dummies(train['TripType'], prefix='TripType_')

In [8]:
train_TT_dummy_df.tail()

,TripType__3,TripType__4,TripType__5,TripType__6,TripType__7,TripType__8,TripType__9,TripType__12,TripType__14,TripType__15,...,TripType__36,TripType__37,TripType__38,TripType__39,TripType__40,TripType__41,TripType__42,TripType__43,TripType__44,TripType__999
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
647052,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

### 2-1-1. Train Weekday

In [9]:
# before
train['Weekday'].value_counts()

Sunday       133975
Saturday     122096
Friday        96247
Monday        83130
Tuesday       72529
Wednesday     71115
Thursday      67962
Name: Weekday, dtype: int64

In [10]:
WD_columns = train['Weekday'].value_counts().index
WD_columns

Index(['Sunday', 'Saturday', 'Friday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'],
      dtype='object')

In [11]:
# after
def weekday_preprocessing(dataframe):
    weekday_dict={
        'Sunday':1,
        'Saturday':2,
        'Friday':3,
        'Monday':4,
        'Tuesday':5,
        'Wednesday':6,
        'Thursday':7,
    }
    return dataframe['Weekday'].map(weekday_dict)

train['Weekday'] = weekday_preprocessing(train)
train['Weekday'].value_counts()

1    133975
2    122096
3     96247
4     83130
5     72529
6     71115
7     67962
Name: Weekday, dtype: int64

In [12]:
train_WD_dummy_df = pd.get_dummies(train['Weekday'])
train_WD_dummy_df.columns = WD_columns

In [13]:
train_WD_dummy_df.tail()

,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday
647049,1,0,0,0,0,0,0
647050,1,0,0,0,0,0,0
647051,1,0,0,0,0,0,0
647052,1,0,0,0,0,0,0
647053,1,0,0,0,0,0,0


### 2-1-2. Test Weekday

In [6]:
# before
test['Weekday'].value_counts()

Sunday       133233
Saturday     123974
Friday        94764
Monday        85174
Tuesday       73742
Wednesday     72481
Thursday      70278
Name: Weekday, dtype: int64

In [7]:
WD_columns = test['Weekday'].value_counts().index
WD_columns

Index(['Sunday', 'Saturday', 'Friday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'],
      dtype='object')

In [8]:
# after
def weekday_preprocessing(dataframe):
    weekday_dict={
        'Sunday':1,
        'Saturday':2,
        'Friday':3,
        'Monday':4,
        'Tuesday':5,
        'Wednesday':6,
        'Thursday':7,
    }
    return dataframe['Weekday'].map(weekday_dict)

test['Weekday'] = weekday_preprocessing(test)
test['Weekday'].value_counts()

1    133233
2    123974
3     94764
4     85174
5     73742
6     72481
7     70278
Name: Weekday, dtype: int64

In [9]:
test_WD_dummy_df = pd.get_dummies(test['Weekday'])
test_WD_dummy_df.columns = WD_columns

In [10]:
test_WD_dummy_df.tail()

,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday
653641,1,0,0,0,0,0,0
653642,1,0,0,0,0,0,0
653643,1,0,0,0,0,0,0
653644,1,0,0,0,0,0,0
653645,1,0,0,0,0,0,0


---

### 2-2-1. Train Upc

In [25]:
train_upc_count_df = pd.DataFrame(train['Upc'].value_counts()).reset_index()
train_upc_count_df.columns=['Upc','Upc count']
train_upc_count_df.head()

,Upc,Upc count
0,4.011000e+03,9746
1,6.053886e+10,2506
2,7.874235e+09,2070
3,7.874235e+09,1892
4,4.046000e+03,1654


In [26]:
train_upc_count_df['Upc count'][:20000].sum()/train_upc_count_df['Upc count'].sum()

0.7438405449931536

In [16]:
train_upc_top20000_columns = train['Upc'][:20000].unique()

In [17]:
test_upc_top20000_columns = test['Upc'][:20000].unique()

In [18]:
in_train_not_in_test = [i for i in train_upc_top20000_columns if i not in test_upc_top20000_columns]

In [19]:
len(in_train_not_in_test)

8252

In [31]:
train_upc_top20000_dummy_df = pd.get_dummies(train['Upc'][:20000])

In [32]:
train_upc_top20000_dummy_df.drop(columns=in_train_not_in_test,inplace=True)

In [33]:
train_upc_top20000_dummy_df.columns = ['Upc_'+str(col) for col in train_upc_top20000_dummy_df.columns]
train_upc_top20000_dummy_df.tail()

,Upc_3082.0,Upc_3107.0,Upc_3112.0,Upc_3151.0,Upc_3207.0,Upc_3283.0,Upc_3421.0,Upc_4011.0,Upc_4016.0,Upc_4017.0,...,Upc_880614469368.0,Upc_880614469376.0,Upc_880614469572.0,Upc_880614469573.0,Upc_880615608239.0,Upc_880632366366.0,Upc_880632366552.0,Upc_932110480262.0,Upc_978163109762.0,Upc_978163346252.0
19995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2-2-2. Test Upc

In [11]:
test_upc_count_df = pd.DataFrame(test['Upc'].value_counts()).reset_index()
test_upc_count_df.columns=['Upc','Upc count']
test_upc_count_df.head()

,Upc,Upc count
0,4.011000e+03,9548
1,6.053886e+10,2472
2,7.874235e+09,2074
3,7.874235e+09,1901
4,4.046000e+03,1668


In [12]:
test_upc_count_df['Upc count'][:20000].sum()/test_upc_count_df['Upc count'].sum()

0.7428240974472421

In [20]:
in_test_not_in_train = [i for i in test_upc_top20000_columns if i not in train_upc_top20000_columns]

In [21]:
len(in_test_not_in_train)

8215

In [22]:
test_upc_top20000_dummy_df = pd.get_dummies(test['Upc'][:20000])

In [23]:
test_upc_top20000_dummy_df.drop(columns=in_test_not_in_train,inplace=True)

In [24]:
test_upc_top20000_dummy_df.columns = ['Upc_'+str(col) for col in test_upc_top20000_dummy_df.columns]
test_upc_top20000_dummy_df.tail()

,Upc_3082.0,Upc_3107.0,Upc_3112.0,Upc_3151.0,Upc_3207.0,Upc_3283.0,Upc_3421.0,Upc_4011.0,Upc_4016.0,Upc_4017.0,...,Upc_880614469368.0,Upc_880614469376.0,Upc_880614469572.0,Upc_880614469573.0,Upc_880615608239.0,Upc_880632366366.0,Upc_880632366552.0,Upc_932110480262.0,Upc_978163109762.0,Upc_978163346252.0
19995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

### 2-3-1. Train DepartmentDescription

In [41]:
print(len(train['DepartmentDescription'].sort_values().unique()))
train['DepartmentDescription'].sort_values().unique()

67


array(['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY',
       'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES',
       'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES',
       'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD',
       'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY',
       'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES',
       'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14',
       'GROCERY DRY GOODS', 'HARDWARE', 'HEALTH AND BEAUTY AIDS',
       'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS',
       'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS',
       'IMPULSE MERCHANDISE', 'INFANT APPAREL',
       'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES',
       'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS',
       'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN',
       'MEDIA AND GAMING', 'MENS WEAR', 'OFFICE SUPPLIES',
       'OPTICAL - FRAMES', 'OPTICAL - LENSES', '

In [42]:
train_DD_dummy_df = pd.get_dummies(train['DepartmentDescription'])
train_DD_dummy_df.tail()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# train_DD_dummy_df_columns=[]
# for idx in preprocessing.LabelEncoder().fit_transform(train_DD_dummy_df.columns):
#     train_DD_dummy_df_columns.append('DD_{}'.format(idx))
# train_DD_dummy_df.columns = train_DD_dummy_df_columns
# train_DD_dummy_df.tail()

### 2-3-2. Test DepartmentDescription

In [25]:
print(len(test['DepartmentDescription'].sort_values().unique()))
test['DepartmentDescription'].sort_values().unique()

66


array(['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY',
       'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES',
       'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES',
       'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD',
       'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY',
       'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES',
       'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14',
       'GROCERY DRY GOODS', 'HARDWARE', 'HOME DECOR', 'HOME MANAGEMENT',
       'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP',
       'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL',
       'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES',
       'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS',
       'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN',
       'MEDIA AND GAMING', 'MENS WEAR', 'OFFICE SUPPLIES',
       'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS',
       'PAINT

'HEALTH AND BEAUTY AIDS' DD가 test에는 없다.!!!

In [26]:
test_DD_dummy_df = pd.get_dummies(test['DepartmentDescription'])
test_DD_temp_dummy_df = pd.concat([test_DD_dummy_df.loc[:,test_DD_dummy_df.columns[:26]], 
                                   pd.DataFrame([0]*len(test), columns=['HEALTH AND BEAUTY AIDS'])], axis=1)
test_DD_dummy_df = pd.concat([test_DD_temp_dummy_df,
                              test_DD_dummy_df.loc[:,test_DD_dummy_df.columns[26:]]],axis=1)
test_DD_dummy_df.tail()

,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
653641,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
# test_DD_dummy_df_columns=[]
# for idx in preprocessing.LabelEncoder().fit_transform(test_DD_dummy_df.columns):
#     test_DD_dummy_df_columns.append('DD_{}'.format(idx))
# test_DD_dummy_df.columns = test_DD_dummy_df_columns
# test_DD_dummy_df.tail()

### 2-4-1. Train FinelineNumber

In [31]:
len(train['FinelineNumber'].sort_values().unique())

5195

In [32]:
test_FLN_columns = test['FinelineNumber'].sort_values().unique()
train_FLN_columns = train['FinelineNumber'].sort_values().unique()

In [33]:
# in Test, not in Train
in_test_not_in_train = \
[i for i in test_FLN_columns if i not in train_FLN_columns]

In [34]:
len(in_test_not_in_train)

158

In [35]:
train_FLN_dummy_df = pd.get_dummies(train['FinelineNumber'], prefix='FLN_')
train_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN__9964.0,FLN__9966.0,FLN__9967.0,FLN__9970.0,FLN__9971.0,FLN__9974.0,FLN__9975.0,FLN__9991.0,FLN__9997.0,FLN__9998.0
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
%%time

FLN_zero_series = pd.Series([0]*len(train))
train_FLN_zero_dummy_df = pd.concat([FLN_zero_series, FLN_zero_series],axis=1)
for _ in range(6):
    train_FLN_zero_dummy_df = pd.concat([train_FLN_zero_dummy_df, train_FLN_zero_dummy_df],axis=1)

for _ in range(30):
    train_FLN_zero_dummy_df = pd.concat([train_FLN_zero_dummy_df, FLN_zero_series],axis=1)

CPU times: user 21 s, sys: 19.2 s, total: 40.2 s
Wall time: 40.4 s


In [53]:
train_FLN_zero_dummy_df.columns = ['FLN_'+str(col) for col in in_test_not_in_train]
train_FLN_zero_dummy_df.tail()

,FLN_92.0,FLN_186.0,FLN_187.0,FLN_480.0,FLN_489.0,FLN_546.0,FLN_573.0,FLN_646.0,FLN_678.0,FLN_743.0,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
%time train_all_FLN_dummy_df = pd.concat([train_FLN_dummy_df, train_FLN_zero_dummy_df], axis=1)

CPU times: user 42.1 s, sys: 4.46 s, total: 46.6 s
Wall time: 47.2 s


In [55]:
train_all_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2-4-2. Test FinelineNumber

In [36]:
len(test['FinelineNumber'].sort_values().unique())

5203

In [37]:
test_FLN_dummy_df = pd.get_dummies(test['FinelineNumber'], prefix='FLN_')
test_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN__9967.0,FLN__9969.0,FLN__9970.0,FLN__9971.0,FLN__9974.0,FLN__9975.0,FLN__9991.0,FLN__9997.0,FLN__9998.0,FLN__9999.0
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# in Train, not in Test
in_train_not_in_test = \
[i for i in train_FLN_columns if i not in test_FLN_columns]

In [39]:
len(in_train_not_in_test)

150

In [40]:
%%time

FLN_zero_series = pd.Series([0]*len(test))
test_FLN_zero_dummy_df = pd.concat([FLN_zero_series, FLN_zero_series],axis=1)
for _ in range(6):
    test_FLN_zero_dummy_df = pd.concat([test_FLN_zero_dummy_df, test_FLN_zero_dummy_df],axis=1)

for _ in range(22):
    test_FLN_zero_dummy_df = pd.concat([test_FLN_zero_dummy_df, FLN_zero_series],axis=1)

CPU times: user 16.8 s, sys: 17.6 s, total: 34.4 s
Wall time: 35.7 s


In [41]:
test_FLN_zero_dummy_df.columns = ['FLN_'+str(col) for col in in_train_not_in_test]
test_FLN_zero_dummy_df.tail()

,FLN_45.0,FLN_238.0,FLN_245.0,FLN_332.0,FLN_373.0,FLN_524.0,FLN_551.0,FLN_567.0,FLN_593.0,FLN_830.0,...,FLN_8801.0,FLN_8903.0,FLN_8930.0,FLN_8938.0,FLN_9175.0,FLN_9527.0,FLN_9792.0,FLN_9937.0,FLN_9942.0,FLN_9964.0
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
%time test_all_FLN_dummy_df = pd.concat([test_FLN_dummy_df, test_FLN_zero_dummy_df], axis=1)

CPU times: user 42.3 s, sys: 4.97 s, total: 47.3 s
Wall time: 47.8 s


In [43]:
test_all_FLN_dummy_df.tail()

,FLN__0.0,FLN__1.0,FLN__2.0,FLN__3.0,FLN__4.0,FLN__5.0,FLN__6.0,FLN__7.0,FLN__8.0,FLN__9.0,...,FLN_8801.0,FLN_8903.0,FLN_8930.0,FLN_8938.0,FLN_9175.0,FLN_9527.0,FLN_9792.0,FLN_9937.0,FLN_9942.0,FLN_9964.0
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

### 2-5-1. Train ScanCount

In [64]:
SC_in_test_not_in_train = \
[i for i in test['ScanCount'].sort_values().unique() if i not in train['ScanCount'].sort_values().unique()]

In [65]:
SC_in_test_not_in_train

[-21, -14, -11, -8, 26, 27, 28, 29, 35, 36, 40, 41, 42, 45, 55]

In [66]:
train_SC_dummy_df = pd.get_dummies(train['ScanCount'],prefix='SC_')

In [67]:
train_SC_dummy_df.tail()

,SC__-12,SC__-10,SC__-9,SC__-7,SC__-6,SC__-5,SC__-4,SC__-3,SC__-2,SC__-1,...,SC__20,SC__22,SC__23,SC__24,SC__25,SC__30,SC__31,SC__46,SC__51,SC__71
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
SC_zero_series = pd.Series([0]*len(train))
train_SC_zero_dummy_df = pd.concat([SC_zero_series, SC_zero_series],axis=1)
for _ in range(13):
    train_SC_zero_dummy_df = pd.concat([train_SC_zero_dummy_df, SC_zero_series],axis=1)

In [69]:
train_SC_zero_dummy_df.columns = ['SC_'+str(col) for col in SC_in_test_not_in_train]
train_SC_zero_dummy_df.tail()

,SC_-21,SC_-14,SC_-11,SC_-8,SC_26,SC_27,SC_28,SC_29,SC_35,SC_36,SC_40,SC_41,SC_42,SC_45,SC_55
647049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [70]:
train_all_SC_dummy_df = pd.concat([train_SC_dummy_df, train_SC_zero_dummy_df], axis=1)

In [71]:
train_all_SC_dummy_df.tail()

,SC__-12,SC__-10,SC__-9,SC__-7,SC__-6,SC__-5,SC__-4,SC__-3,SC__-2,SC__-1,...,SC_27,SC_28,SC_29,SC_35,SC_36,SC_40,SC_41,SC_42,SC_45,SC_55
647049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647051,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647052,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
647053,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2-5-2. Test ScanCount

In [44]:
SC_in_train_not_in_test = \
[i for i in train['ScanCount'].sort_values().unique() if i not in test['ScanCount'].sort_values().unique()]

In [45]:
SC_in_train_not_in_test

[-12, 25, 30, 51, 71]

In [46]:
test_SC_dummy_df = pd.get_dummies(test['ScanCount'],prefix='SC_')

In [47]:
test_SC_dummy_df.tail()

,SC__-21,SC__-14,SC__-11,SC__-10,SC__-9,SC__-8,SC__-7,SC__-6,SC__-5,SC__-4,...,SC__29,SC__31,SC__35,SC__36,SC__40,SC__41,SC__42,SC__45,SC__46,SC__55
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
SC_zero_series = pd.Series([0]*len(test))
test_SC_zero_dummy_df = pd.concat([SC_zero_series, SC_zero_series],axis=1)
for _ in range(3):
    test_SC_zero_dummy_df = pd.concat([test_SC_zero_dummy_df, SC_zero_series],axis=1)

In [49]:
test_SC_zero_dummy_df.columns = ['SC_'+str(col) for col in SC_in_train_not_in_test]
test_SC_zero_dummy_df.tail()

,SC_-12,SC_25,SC_30,SC_51,SC_71
653641,0,0,0,0,0
653642,0,0,0,0,0
653643,0,0,0,0,0
653644,0,0,0,0,0
653645,0,0,0,0,0


In [50]:
test_all_SC_dummy_df = pd.concat([test_SC_dummy_df, test_SC_zero_dummy_df], axis=1)

In [51]:
test_all_SC_dummy_df.tail()

,SC__-21,SC__-14,SC__-11,SC__-10,SC__-9,SC__-8,SC__-7,SC__-6,SC__-5,SC__-4,...,SC__41,SC__42,SC__45,SC__46,SC__55,SC_-12,SC_25,SC_30,SC_51,SC_71
653641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
653645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
myslack_incomming.send_slack('Continue encoding work ')

<Response [200]>


---

## Preprocessing Step3 - Concat VisitNumber & all Dummy DFs

### Train

- VisitNumber & Weekday

In [81]:
%time train_concated_df = pd.concat([train['VisitNumber'],train_WD_dummy_df],axis=1)

CPU times: user 8.08 ms, sys: 12.2 ms, total: 20.3 ms
Wall time: 30.2 ms


- append Upc

In [82]:
%time train_concated_df = pd.concat([train_concated_df,train_upc_top20000_dummy_df],axis=1)

CPU times: user 1min 13s, sys: 51.6 s, total: 2min 5s
Wall time: 2min 15s


- append ScanCount

In [83]:
%time train_concated_df = pd.concat([train_concated_df,train_all_SC_dummy_df],axis=1)

CPU times: user 11.1 s, sys: 29.9 s, total: 41 s
Wall time: 47.3 s


- append DepartmentDescription

In [84]:
%time train_concated_df = pd.concat([train_concated_df,train_DD_dummy_df],axis=1)

CPU times: user 10.8 s, sys: 28.9 s, total: 39.6 s
Wall time: 47.2 s


- append FinelineNumber

In [85]:
%time train_concated_df = pd.concat([train_concated_df,train_all_FLN_dummy_df],axis=1)

CPU times: user 12.8 s, sys: 32.2 s, total: 44.9 s
Wall time: 52 s


In [86]:
train_concated_df.shape

(647054, 9620)

In [87]:
train_concated_df.tail()

,VisitNumber,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday,Upc_3082.0,Upc_3107.0,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
647049,191346,1,0,0,0,0,0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
647050,191346,1,0,0,0,0,0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
647051,191346,1,0,0,0,0,0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
647052,191347,1,0,0,0,0,0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
647053,191347,1,0,0,0,0,0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [88]:
%time final_train = train_concated_df.groupby('VisitNumber').sum()

CPU times: user 2min 59s, sys: 2min 46s, total: 5min 46s
Wall time: 6min 31s


In [90]:
final_train.shape

(95674, 9619)

- groupby TripType

In [91]:
%time train_groupbyed_TripType = train.groupby('VisitNumber')['TripType'].agg(lambda x: scipy.stats.mode(x)[0])

CPU times: user 6.97 s, sys: 139 ms, total: 7.11 s
Wall time: 7.21 s


In [92]:
len(train_groupbyed_TripType)

95674

In [93]:
%time final_train = pd.concat([train_groupbyed_TripType,final_train],axis=1)

CPU times: user 1.67 s, sys: 1.45 s, total: 3.12 s
Wall time: 3.14 s


In [94]:
final_train.tail()

,TripType,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday,Upc_3082.0,Upc_3107.0,...,FLN_9817.0,FLN_9818.0,FLN_9865.0,FLN_9894.0,FLN_9895.0,FLN_9916.0,FLN_9940.0,FLN_9945.0,FLN_9969.0,FLN_9999.0
VisitNumber,,,,,,,,,,,,,,,,,,,,,
191343,25,7,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
191344,22,5,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
191345,39,13,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
191346,39,17,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
191347,8,2,0,0,0,0,0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
final_train_columns_length = len(final_train.columns)
final_train_columns_length

9620

In [102]:
# col_1 = final_train.columns[:int(final_train_columns_length/2)]
# col_2 = final_train.columns[int(final_train_columns_length/2):]
# final_train_1 = final_train.loc[:,col_1]
# final_train_2 = final_train.loc[:,col_2]

In [99]:
%time final_train.to_csv('./data/final_train_Ver2.csv')

CPU times: user 12min 45s, sys: 32.9 s, total: 13min 18s
Wall time: 13min 20s


In [100]:
myslack_incomming.send_slack('final_train_Ver2.csv maden')

<Response [200]>


In [74]:
# %%time 
# pickle.dump(final_train, open("./data/final_train_Ver2.pkl", "wb"))

### test

- VisitNumber & Weekday

In [52]:
%time test_concated_df = pd.concat([test['VisitNumber'],test_WD_dummy_df],axis=1)

CPU times: user 6.09 ms, sys: 1.34 ms, total: 7.43 ms
Wall time: 6.09 ms


- append Upc

In [53]:
%time test_concated_df = pd.concat([test_concated_df,test_upc_top20000_dummy_df],axis=1)

CPU times: user 1min 10s, sys: 48.9 s, total: 1min 58s
Wall time: 2min 12s


- append ScanCount

In [54]:
%time test_concated_df = pd.concat([test_concated_df,test_all_SC_dummy_df],axis=1)

CPU times: user 10.5 s, sys: 27.3 s, total: 37.7 s
Wall time: 42.6 s


- append DepartmentDescription

In [55]:
%time test_concated_df = pd.concat([test_concated_df,test_DD_dummy_df],axis=1)

CPU times: user 10.6 s, sys: 27.2 s, total: 37.8 s
Wall time: 42.6 s


- append FinelineNumber

In [56]:
%time test_concated_df = pd.concat([test_concated_df,test_all_FLN_dummy_df],axis=1)

CPU times: user 12.6 s, sys: 31 s, total: 43.6 s
Wall time: 49.5 s


In [57]:
test_concated_df.shape

(653646, 9620)

In [58]:
%time final_test = test_concated_df.groupby('VisitNumber').sum()

CPU times: user 2min 55s, sys: 2min 39s, total: 5min 34s
Wall time: 6min 14s


In [59]:
final_test.shape

(95674, 9619)

In [60]:
final_test.tail()

,Sunday,Saturday,Friday,Monday,Tuesday,Wednesday,Thursday,Upc_3082.0,Upc_3107.0,Upc_3112.0,...,FLN_8801.0,FLN_8903.0,FLN_8930.0,FLN_8938.0,FLN_9175.0,FLN_9527.0,FLN_9792.0,FLN_9937.0,FLN_9942.0,FLN_9964.0
VisitNumber,,,,,,,,,,,,,,,,,,,,,
191338,7,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
191339,5,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
191340,1,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
191341,10,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
191348,7,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
%%time
final_test.to_csv('./data/final_test_Ver2.csv')

CPU times: user 12min 26s, sys: 36.6 s, total: 13min 2s
Wall time: 23min 26s


In [73]:
# %%time 
# pickle.dump(final_test, open("./data/final_test_ver2.pkl", "wb"))

In [63]:
myslack_incomming.send_slack('All Encoding work finished!!!')

<Response [200]>


### 데이터 pickle로 저장하기

In [2]:
%time train=pd.read_csv('./data/final_train_Ver2.csv')

CPU times: user 4min 16s, sys: 29 s, total: 4min 45s
Wall time: 4min 47s


In [3]:
%time test=pd.read_csv('./data/final_test_Ver2.csv')

CPU times: user 4min 8s, sys: 26.4 s, total: 4min 35s
Wall time: 19min 24s


In [6]:
myslack_incomming.send_slack('파일 다 읽었다')

<Response [200]>


In [28]:
%time pickle.dump(train, open("./data/Ver2_train.plk", "wb"))

OSError: [Errno 22] Invalid argument

In [7]:
train_half_columns_1 = train.columns[:4810]
train_half_columns_2 = train.columns[4810:]

In [8]:
train_half_df_1 = train.loc[:,train_half_columns_1]
train_half_df_2 = train.loc[:,train_half_columns_2]

In [25]:
# %time pickle.dump(train_half_df_1, open("./data/Ver2_train_half_df_1.plk", "wb"))

In [ ]:
# %time pickle.dump(train_half_df_2, open("./data/Ver2_train_half_df_2.pkl", "wb"))

In [26]:
# %time final_test_ver2 = pickle.load(open("./data/final_test_ver2.pkl", "rb"))

---